# CAV project demo
Main demo for the CAV project
- object detection and tracking
- maps display
- BSM message broadcasting


In [ ]:
import cv2
import glob
import os
import sys
import numpy as np

from random import randint
import matplotlib.pyplot as plt

import socket
import json

from time import time
from cav.objects import Object, BoundingBox, ObjectType
from cav.parameters import Parameters

from cav.visualization import Map, plotBoxes, bsmImg


# Deep sort imports
from deep_sort import nn_matching
from deep_sort.tracker import Tracker
from deep_sort.detection import Detection
%matplotlib inline 

### BSM Server configuration

In [ ]:
PUSH_BSM = True # If true, bsms are being pushed to the server

HOST = '127.0.0.1'   # The server's hostname or IP address
PORT = 65432        # The port used by the server
DATA_BUFF = 4096

In [ ]:
# FUnction that sends data
def sendBsm(s, bsm):
    data = {
        'mode' : 'push',
        'msg' : bsm
    }
    
    msg = json.dumps(data)
    msg = str.encode(msg)
    s.sendall(msg)
    data = s.recv(1024)  
    return data    

## Deep sort

In [ ]:
from helper import ImageEncoder, create_box_encoder, extract_image_patch

In [ ]:
ENCODER_PATH = "./deep_sort_network/mars-small128.pb"
ENCODER_BATCH_SIZE = 32
ENCODER_INPUT_NAME = "images"
ENCODER_OUTPUT_NAME = "features"

image_encoder = ImageEncoder(ENCODER_PATH, ENCODER_INPUT_NAME, ENCODER_OUTPUT_NAME)
encoder = create_box_encoder(ENCODER_PATH, batch_size=32)

## Create tracker

In [ ]:
max_cosine_distance = 0.2
nn_budget = 100

metric = nn_matching.NearestNeighborDistanceMetric(
    "cosine", max_cosine_distance, nn_budget)

### Create object detector

In [ ]:
from cav.detection import ObjectDetector
import cv2
MODEL_PATH = '../../CAV/models/frcnn/inference/saved_model/'
od = ObjectDetector(MODEL_PATH)

### Create params and map objects

In [ ]:
params = Parameters()
params.generateParameters('./config/params.json')
mymap = Map('./images/KavehWindowViewSatelite.jpg', './config/icons.json', params)
plt.imshow(mymap.getMap(), interpolation='nearest')
plt.show()

#### Logging parameters

In [ ]:
PREPARE_VISUALIZATION = False ### Temporary and ugly - Should be cleaned soon
SAVE_FRAMES = False # Doesn't work yet, an idea how to correct PREPARE_VISUALIZATION
SAVE_VIDEO = False # Doesn't work yet, an idea how to correct PREPARE_VISUALIZATION

SAVE_LOG = None #### Saves logs with all detected objects (path to file or none)
SAVE_LOG = 'test_log_20201028.csv'
SAVE_EMPTY_FRAMES = None ### Folder where empty frames shall be saved
SAVE_EMPTY_FRAMES = './empty_frames'


#### Other parameters

In [ ]:
VIDEO_X = 720
VIDEO_Y = 480
FRAMES_SEC = 15

MAX_BOXES_TO_DRAW = 20
MIN_SCORE_THRESH = 0.5
IOU_COMMON_THRESHOLD = 0.50
NOT_DETECTED_TRHESHOLD = 1

MAPSHAPE = mymap.getMap().shape
print ('Y dimension of map is {:.3f} larger than Y dimension of the video'
      .format(MAPSHAPE[0] / VIDEO_Y))

MAP_RESIZE = 2

print ('Y dimension of map is {:.3f} larger than Y dimension of the video. Size of the map is reduced {} times.'
      .format(MAPSHAPE[0] / VIDEO_Y, MAP_RESIZE))

BSM_X = 0

FINAL_X = VIDEO_X + int(MAPSHAPE[1] / MAP_RESIZE) + BSM_X
FINAL_Y = max(VIDEO_Y, int(MAPSHAPE[0] / MAP_RESIZE))

print ('Video size: [{}, {}], Final size: [{}, {}]'
      .format(VIDEO_X, VIDEO_Y, FINAL_X, FINAL_Y))

RESIZE = False

In [ ]:
print (MAPSHAPE[0] / VIDEO_Y)

In [ ]:
#cap = cv2.VideoCapture('rtmp://170.93.143.157/rtplive/c5ff65d2008900a1004f823633235daa')
#cap = cv2.VideoCapture('/data/CAV/VideoFootage/Corona 24H cam1.avi')
cap = cv2.VideoCapture() 
cap.open("rtsp://128.8.215.215/1")

fourcc = cv2.VideoWriter_fourcc(*'XVID')

prepare_visualization_base = PREPARE_VISUALIZATION

if PREPARE_VISUALIZATION:
    if RESIZE:
        out = cv2.VideoWriter('Test1_output.avi',fourcc, 2*FRAMES_SEC, (int(FINAL_X/2), int(FINAL_Y/2)))
    else:
        out = cv2.VideoWriter('Test1_output.avi',fourcc, 2*FRAMES_SEC, (int(FINAL_X), int(FINAL_Y)))
else:
    out = None


objects = []

results = []
colors = {}


tracker = Tracker(metric)

i = 0
t = time()

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    if PUSH_BSM:
        s.connect((HOST, PORT))
    while cap.isOpened():
        t2 = time() - t
        sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
            i, t2, i/t2))                   
        #sys.stdout.write("Processing Frame {}     \r".format(i))
        ret, image = cap.read()

        boxes, scores, classes = od.detect(image)
        if len(boxes) >= 1:

            boxes_array = [[box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop] for box in boxes]
            boxes_array = np.array(boxes_array)
            bgr_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            features = encoder(bgr_image, boxes_array)
            detections = []

            for box, score, objClass, f_vector in zip(boxes, scores, classes, features):
                detection = Detection(
                    [box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop], #BBox
                    score, f_vector,
                    objClass
                )
                detection.bbox = box
                detections.append(detection)

            tracker.predict()
            tracker.update(detections)                
            
            # Added to save only important frames DELETE LATER
            PREPARE_VISUALIZATION = True
        else:
            tracker.predict()
            PREPARE_VISUALIZATION = prepare_visualization_base
            
        plotboxes = []
        plotcolors = []
        objects = []

        if len(tracker.tracks) >= 1:
            for track in tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue

                if False:
                    bbox = track.to_tlwh()
                    results.append([
                        i, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])


                obj = track.trackedObject

                if obj is not None:
                    plotbox = obj.bboxes[-1]
                    plotbox.trackId = track.track_id
                    plotboxes.append(plotbox)
                    plotcolors.append(obj.color)
                    objects.append(obj)
                    

            if len(plotboxes) >= 1:
                vid = plotBoxes(image, plotboxes, colors=plotcolors)
            else:
                vid = image.copy()
                mapimg = mymap.getMap()

            if PREPARE_VISUALIZATION:
                mapimg = mymap.addObjects(objects)        
        elif PREPARE_VISUALIZATION:
                vid = image.copy()
                mapimg = mymap.getMap()
                    

        # Adding map
        if PREPARE_VISUALIZATION:
            if MAP_RESIZE != 1: # Reducing the size of the map
                width = int(mapimg.shape[1] / MAP_RESIZE)
                height = int(mapimg.shape[0] / MAP_RESIZE)
                dim = (width, height)
                # resize image
                mapimg = cv2.resize(mapimg, dim, interpolation = cv2.INTER_AREA)                   
                    
            final_image = np.zeros((FINAL_Y, FINAL_X,3),dtype=np.uint8)

            final_image[:VIDEO_Y, :VIDEO_X, :] = vid
            final_image[:mapimg.shape[0], VIDEO_X:VIDEO_X + mapimg.shape[1], :] = mapimg  

        if (BSM_X > 0) & PREPARE_VISUALIZATION:
            if len(objects) > 0:
                obj = objects[0]
                color = obj.color
                bsmimg = bsmImg(obj.getBsm(params=params), framecolor=color)
                final_image[:bsmimg.shape[0], VIDEO_X + mapimg.shape[1]:, :] =  bsmimg

            if len(objects) > 1:
                obj = objects[1]
                color = obj.color
                bsmimg = bsmImg(obj.getBsm(params=params), framecolor=color)
                #final_image[bsmimg.shape[0]+10:2*bsmimg.shape[0]+10, VIDEO_X + mapimg.shape[1]:, :] =  bsmimg

        if PREPARE_VISUALIZATION & RESIZE: # Resize image
            width = int(final_image.shape[1] / 2)
            height = int(final_image.shape[0] / 2)
            dim = (width, height)
            # resize image
            final_image = cv2.resize(final_image, dim, interpolation = cv2.INTER_AREA)

        if PREPARE_VISUALIZATION:
            cv2.imwrite('./tmp/im_{}.jpg'.format(str(i).zfill(4)), final_image)
            if out is not None:
                out.write(final_image)            
                
            
            
        bsm_list = []
        
        if len(objects) > 0:
            for obj in objects:
                bsm_list.append(obj.getBsm(retDic = False, params=params, roundValues = True, includeNone = False))

            #print (bsm_list)
            if PUSH_BSM:
                data = sendBsm(s, json.dumps(bsm_list))
                #print ("Response: {}\n".format(data))            

                
            if SAVE_EMPTY_FRAMES is not None:
                cv2.imwrite('{}/im_{}.jpg'.format(SAVE_EMPTY_FRAMES, str(i).zfill(4)), image)
                
            if SAVE_LOG is not None:
                logfile = open('./logs/{}'.format(SAVE_LOG, 'w'))
                for obj in objects:
                    line = '{},{},{}'.format(i,time(),obj.getParams(asCsv=true))                               
                    print(line,file=logfile)                    
                                
        i = i+1
        if False: #i > 5* FRAMES_SEC:
            if False:
                f = open('tmp_tracking_results_demo.txt', 'w')
                for row in results:
                    print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1' % (
                        row[0], row[1], row[2], row[3], row[4], row[5]),file=f)        
            break
                  
            
                        
t = time() - t                             
print('\n\n{} frames done in {:.1f} seconds ({:.2f} frames/sec)'.format(
    i, t, i/t))                             
cap.release()
    


# Generate Video

In [ ]:
from helper import generate_video
generate_video(fps = 30)